# 模型评估与选择

## 经验误差与过拟合

**错误率（error rate）**：分类错误的样本数占样本总数的比例。

如果在m个样本中有a个样本分类错误，则错误率 $E = \frac{a}{m} $

**精度（accuracy）**：精度=1-错误率。

若错误率为$E = \frac{a}{m} $，那么精度为 $A = 1 - \frac{a}{m} $。

**误差（error）**：学习器的预测输出 $y^{'}$ 与样本的真实输出 y 之间的差异$error = y - y^{'} $。

**训练误差（training error）**：也称**经验误差（empirical error）**，即学习器在训练集上的误差$error = y - y^{'}|_{(x \in 训练集)} $。

**泛化误差（generalization error）**：学习器在新样本上的误差$error = y - y^{'}|_{(x \in 测试集)} $。

显然，我们希望得到泛化误差小的学习器。然而，我们不知道新数据是什么样子，只能尽力使经验误差最小化。

**过拟合（overfitting）**：有时我们把学习器训练的太好，其训练误差非常小，但其泛化性能很差，即泛化误差很大，这种情况称为过拟合。

**欠拟合（underfitting）**：指训练样本的一般性质尚未学好，训练误差和泛化误差都较高，泛化能力也表现一般。

下面的图片给出了过拟合和欠拟合的对比

![过拟合欠拟合对比](images/02/过拟合欠拟合对比.PNG)



## 模型选择

导致过拟合的原因较多，最常见的是学习器能力过强，以至于把训练样本所包含的不太一般性的特性都学到了；而欠拟合则通常是由于学习能力低下造成的。

欠拟合容易克服，而过拟合则不易解决。过拟合是无法彻底避免的，只能缓解。

机器学习问题面临的问题通常是NP难甚至更难，有效的学习算法必然是在多项式时间内运行完成，如果可以彻底避免过拟合，则通过经验误差最小化就能得到最优解，这就意味着我们构造性的证明了“$P = NP$”。**因此只要相信“$P \neq NP$”，过拟合就不可避免。**

在现实问题中，往往有多种学习算法可以使用，甚至对同一个算法，当使用不同参数时也会产生不同的模型。那么应该如何选择呢？这就是**模型选择（model selection）问题**。

理想的解决方案当然是对候选模型的泛化误差进行评估，然后选择泛化误差最小的那个模型。但是，泛化误差无法直接得到，而训练误差又由于过拟合现象的存在而不适合作为标准，那么在现实中如何评估模型与选择模型呢？

这就需要了解一些评估方法。


## 评估方法

通常，我们会通过实验测试来对学习器的泛化误差进行评估，从而选出合适的模型。

为此，需要使用一个“测试集”（testing set）来测试学习器对新样本的判别能力，然后在测试集上的“测试误差”（testing error）作为泛化误差的近似。

![grid_search_workflow](images/02/grid_search_workflow.PNG)


测试集与训练集要尽量互斥，即测试样本尽量不再训练集中出现，未被使用过。

如果我们只有一个包含m个样例的数据集$D = {(x_1,y_1),(x_2,y_2),...,(x_m,y_m)}$,既要训练，又要测试，怎样才能做到呢？

一般有以下几个方法:

- 留出法
- 交叉验证法
- 留一法
- 自助法


### 留出法（hold-out）

直接将数据集D划分为两个互斥的集合，其中一个集合作为训练集S，另一个作为测试集T。

即 $D = S \cup T， S \cap T = \emptyset $。在S上训练出模型后，用T来评估其测试误差，作为对泛化误差的估计。

**需要注意的是，训练集/测试集的划分要尽可能保持数据分布的一致性，避免因数据划分过程引入额外的误差而对最终结果产生影响。** 

例如，在分类任务中至少要保持样本的类别比例相似。这有点像分层采样（stratified sampling）。

**另一个需要注意的是，在给定训练/测试集的样本比例后，仍存在有多种方式对D划分S/T的方法。这会导致使用留出法得到的估计结果往往不够稳定可靠，在使用留出法时，一般要采用若干次随机划分、重复进行实验评估后取平均值作为留出法的评估结果。** 

例如进行100次随机划分，每次产生一个训练/测试集用于实验评估，100次后就得到100个结果，而留出法返回的时这100个结果的平局值。

**常用做法是，使用大约2/3~4/5的样本用于训练，剩余样本用于测试。**

#### 利用sklearn实现留出法验证

在sklearn当中，使用train_test_split可以将数据分为训练集和测试集。

下面使用鸢尾花数据集为例，将其中$\frac{4}{5}$作为训练集,剩下$\frac{1}{5}$作为测试集。

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
print("数据集总样例数与总标记数：",iris.data.shape, iris.target.shape)

X_train, X_test, y_train, y_test = train_test_split(iris.data, 
                                                    iris.target, 
                                                    test_size=0.2, 
                                                    random_state=0)

print("训练集样例数与标记数：",X_train.shape, y_train.shape)
print("测试集样例数与标记数：",X_test.shape, y_test.shape)

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
print("模型评分：", clf.score(X_test, y_test) )

数据集总样例数与总标记数： (150, 4) (150,)
训练集样例数与标记数： (120, 4) (120,)
测试集样例数与标记数： (30, 4) (30,)
模型评分： 1.0


留出法非常的简单。但是存在一些问题，比如有些模型还需要进行超参数评估，这个时候还需要划分一类数据集，叫做验证集。

最后数据集的划分划分变成了这样：

- 训练集，为了进行模型的训练；
- 验证集，为了进行参数的调整；
- 测试集，为了测试模型的好坏。

例如：使用SVM模型时，为了使模型的泛化能力好，需要精心设置超参C。

但是，上面的划分会使训练集变小（仅能反映部分情况，与真实世界越来越远），而且学得的模型会随着我们选择的训练集和验证集不同而不同。

所以这个时候，我们引入了交叉验证（cross-validation 简称cv）。交叉验证仍需要测试集做最后的模型评估，但不再需要验证集。

### 交叉验证法（cross validation）

思路如下：

1.先将数据集D划分为k个大小相似的互斥子集，即$D = D_1 \cup D_2 \cup ...\cup D_k, D_i \cap D_j = \emptyset (i \neq j)$。

每个子集$D_i$都尽可能保持数据分布的一致性，即从D中通过分层采样得到。

2.然后，每次用k-1个子集的并集作为训练集，余下的那个子集作为测试集。

3.这样就可以获得k组训练/测试集，从而可进行k次训练和测试。

4.最终，返回的是这k个测试结果的均值。

![k折交叉验证的数据划分](images/02/k折交叉验证的数据划分.PNG)


显然，交叉验证法评估结果的稳定性和保真性，在很大程度上，取决于k的取值。所以交叉验证法也称为**k着交叉验证法（k-fold cross validation）**。

最常见的k取值为 $k=10$，此时称为10折交叉验证，其他常用的有5、20等。

与留出法相似，将数据集D划分为k个子集同样存在多种划分方式。为减小因样本划分不同而引入的差别，**k折交叉验证通常要随机使用不同的划分，重复$p$次**，最终的评估结果是这$p$次$k$折交叉验证结果的均值，例如常见的有“10次10折交叉验证法”。

k-折交叉验证得出的性能指标是循环计算中每个值的平均值。 该方法虽然计算代价很高，但是它不会浪费太多的数据，在处理样本数据集较少的问题时比较有优势。

#### sklearn中的交叉验证方法

下面的sklearn数据划分工具，假定数据是独立相同分布的（所有的样本来源于相同的生成过程，且生成过程无记忆）。

> 独立同分布数据在实践中很少成立。如果样本是使用时间相关的过程生成的，则使用 time-series aware cross-validation scheme 更安全。 同样，如果我们知道生成过程具有 group structure （群体结构）（从不同 subjects（主体） ， experiments（实验）， measurement devices （测量设备）收集的样本），则使用 group-wise cross-validation 更安全。


**KFold**

KFold 将所有的样例划分为K个折叠（组）。预测函数学习时使用 K-1个折叠中的数据，最后1个剩下的折叠会用于测试。

每个折叠由两个部分组成，第一个作为训练集 ，另一个作为测试。

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
print("数据集总样例数与总标记数：",iris.data.shape, iris.target.shape)

kf = KFold(n_splits=10)
clf = svm.SVC(kernel='linear', C=1)
for k, (train, test) in enumerate(kf.split(iris.data, iris.target)):  
    #print("[第{}折，训练集{}，测试集{}] ".format(k,train,test ))
    clf.fit(iris.data[train], iris.target[train])
    print("第{}折.模型评分：{}".format(k,clf.score(iris.data[test], iris.target[test])) )



数据集总样例数与总标记数： (150, 4) (150,)
第0折.模型评分：1.0
第1折.模型评分：1.0
第2折.模型评分：1.0
第3折.模型评分：1.0
第4折.模型评分：0.8666666666666667
第5折.模型评分：0.9333333333333333
第6折.模型评分：1.0
第7折.模型评分：1.0
第8折.模型评分：0.8666666666666667
第9折.模型评分：0.9333333333333333


##### 计算交叉验证的指标

使用交叉验证最简单的方法是在估计器和数据集上调用 cross_val_score 辅助函数。

cross_val_score对数据集进行指定次数的交叉验证并为每次验证效果评测

下面的例子展示了如何使用10折交叉验证，并记录了各次的评分(10折交叉验证评分的平均分和 95% 置信区间):

In [21]:
from sklearn.model_selection import cross_val_score

clist = np.linspace(start=0.1, stop=2, num=20)#产生从start=-5到stop=5的等差数列，共产生21个
print("候选的超参C：",clist)

for c in clist:
    clf = svm.SVC(kernel='linear', C=c)
    scores = cross_val_score(clf, iris.data, iris.target, cv=10)
    print("超参C={}时，精度: {:4f} (+/- {:.2f})".format(c,scores.mean(), scores.std() * 2))                             

候选的超参C： [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8
 1.9 2. ]
超参C=0.1时，精度: 0.973333 (+/- 0.07)
超参C=0.2时，精度: 0.980000 (+/- 0.06)
超参C=0.3时，精度: 0.980000 (+/- 0.06)
超参C=0.4时，精度: 0.986667 (+/- 0.05)
超参C=0.5时，精度: 0.986667 (+/- 0.05)
超参C=0.6时，精度: 0.980000 (+/- 0.06)
超参C=0.7时，精度: 0.986667 (+/- 0.05)
超参C=0.7999999999999999时，精度: 0.986667 (+/- 0.05)
超参C=0.8999999999999999时，精度: 0.980000 (+/- 0.09)
超参C=0.9999999999999999时，精度: 0.973333 (+/- 0.09)
超参C=1.0999999999999999时，精度: 0.980000 (+/- 0.06)
超参C=1.2时，精度: 0.980000 (+/- 0.06)
超参C=1.3时，精度: 0.980000 (+/- 0.06)
超参C=1.4时，精度: 0.986667 (+/- 0.05)
超参C=1.5时，精度: 0.986667 (+/- 0.05)
超参C=1.5999999999999999时，精度: 0.980000 (+/- 0.09)
超参C=1.7时，精度: 0.980000 (+/- 0.09)
超参C=1.8时，精度: 0.980000 (+/- 0.09)
超参C=1.9时，精度: 0.980000 (+/- 0.09)
超参C=2.0时，精度: 0.980000 (+/- 0.09)


注意：默认情况下，交叉验证评分（每个 CV 迭代计算的分数）是sklearn的score方法。如果你对这个评分计算方式有异议，可以通过使用 scoring 参数来改变计算方式。


具体可以参考 https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter 。

下面的例子是使用F1评分公式：

In [26]:
from sklearn import metrics
scores = cross_val_score(clf, iris.data, iris.target, cv=10, scoring='f1_macro')
scores                                              


array([1.        , 0.93265993, 1.        , 1.        , 0.86111111,
       1.        , 0.93265993, 1.        , 1.        , 1.        ])

##### cross_val_predict

cross_val_predict 与cross_val_score 很相像，不过cross_val_predict 返回的是estimator 的分类结果（或回归值）.

这个对于后期模型的改善很重要，可以通过该预测输出对比实际目标值，准确定位到预测出错的地方，为我们参数优化及问题排查十分的重要。

In [14]:
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

predicted = cross_val_predict(clf, iris.data, iris.target, cv=10)

print("预测结果：",predicted)
print("精确度评分：")
metrics.accuracy_score(iris.target, predicted)

预测结果： [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
精确度评分：


0.9733333333333334

如果想采用p次k折交叉验证，可以使用sklearn中的RepeatedKFold：

In [32]:
from sklearn.model_selection import KFold,RepeatedKFold
import numpy as np

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
p = 2
k = 2

kf1 = KFold(n_splits=k)
for train_index, test_index in kf1.split(X):
    print('train_index', train_index, 'test_index', test_index)
    
print('-------'*3)    

kf2 = RepeatedKFold(n_splits=k, n_repeats=p, random_state=0)
for train_index, test_index in kf2.split(X):
    print('train_index', train_index, 'test_index', test_index)

train_index [2 3] test_index [0 1]
train_index [0 1] test_index [2 3]
---------------------
train_index [0 1] test_index [2 3]
train_index [2 3] test_index [0 1]
train_index [1 3] test_index [0 2]
train_index [0 2] test_index [1 3]


### 留一法（Leave One Out，LOO）

假定数据集D中包含m个样本，若令k=m，则得到了交叉验证法的一个特例：留一法。

显然留一法不受随机样本划分方式的影响，因为m个样本只有唯一的方式划分为m个子集，每个子集包含一个样本。

留一法使用的训练集与初始数据集相比只少了一个样本，所以，大多数时候，留一法中北实际评估的模型与期望评估的用D训练初的模型很相似。因此留一法的评估结果往往被认为比较准确。

留一法也有缺陷，在数据集比较大时，训练m个模型的计算开销可能是难以忍受的。

In [29]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
X = np.array([[1, 2], [3, 4]])
y = np.array([1, 2])
loo = LeaveOneOut()
print(loo.get_n_splits(X))#返回拆分迭代次数

for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)

2
TRAIN: [1] TEST: [0]
[[3 4]] [[1 2]] [2] [1]
TRAIN: [0] TEST: [1]
[[1 2]] [[3 4]] [1] [2]


### 自助法（bootstrapping）

我们希望评估的是用D训练出的模型。

但在留出法和交叉验证法中，由于保留了一部分样本用于测试，因此实际评估的模型所使用的训练集比D小，这必然会引入一些因训练样本不同而导致的估计偏差。

**有没有可以减少训练样本规模不同造成的影响，同时还能比较高效地进行实验估计呢？**

**自助法是个较好的解决方案。他直接以自助采样法（bootstrap sampling）为基础。**

给定包含m个样本的数据集D，我们对它进行采样产生数据集$D^{'}$: 

1.每次随机从D中挑选一个样本，将其拷贝放入$D^{'}$；

2.然后再将该样本放回数据集初始数据集D中，使得该样本在下次采样时仍有可能被采到；

3.这个过程重复执行m次后。

我们就得到了包含m个样本的数据集$D^{'}$，这就是自助采样的结果。

显然，D中有一部分样本会在$D^{'}$中多次出现，而另一部分样本不出现。

可以做一个简单估计，样本在m次采样中始终不被采到的概率为：

$(1-\frac{1}{m})^m$，

取极限可以得到：

$\lim_{m \to \infty}(1-\frac{1}{m})^m = \frac{1}{e} \simeq 0.368 $

即通过自助采样，初始数据集D中约有36.8%的样本未出现在采样数据集$D^{'}$中。

于是，我们可以将$D^{'}$作为训练集，$D - D^{'}$作为测试集。

这样，实际评估的模型与期望评估的模型都使用了m个训练样本，而我们仍有数据总量约1/3的没在训练集中出现的样本用于测试。这样的测试结果，亦称“包外估计”（out-of-bag estimate）。

**自助法在数据集较小，难以有效划分训练集/测试集时很有用；**

此外，**自助法能从初始数据集中产生多个不同的训练集，这对集成学习等方法有很大的好处。**

然而，**自助法产生的数据集改变了初始数据集的分布，这会引入估计偏差。**

因此，**在初始数据量足够时，留出法和交叉验证法更常用一些。**


In [34]:
from sklearn.model_selection import LeaveOneOut
import numpy as np
import pandas as pd
import random
data = pd.DataFrame(np.random.rand(10,4),columns=list('ABCD'))
data['y'] = [random.choice([0,1]) for i in range(10)]
print(data)

train = data.sample(frac=1.0,replace=True)
test = data.loc[data.index.difference(train.index)].copy()

print("train set:")
print(train)

print("test set:")
print(test)

          A         B         C         D  y
0  0.139853  0.268120  0.382930  0.303612  0
1  0.968734  0.039626  0.246015  0.922639  1
2  0.644002  0.570469  0.252412  0.993433  0
3  0.350653  0.228341  0.262602  0.720907  1
4  0.967470  0.066834  0.105967  0.373880  0
5  0.299352  0.176775  0.652900  0.036211  1
6  0.363213  0.103385  0.104004  0.106266  1
7  0.613138  0.847137  0.668122  0.882749  1
8  0.792669  0.890965  0.750801  0.382474  1
9  0.051731  0.897289  0.242222  0.564973  0
train set:
          A         B         C         D  y
7  0.613138  0.847137  0.668122  0.882749  1
0  0.139853  0.268120  0.382930  0.303612  0
6  0.363213  0.103385  0.104004  0.106266  1
6  0.363213  0.103385  0.104004  0.106266  1
2  0.644002  0.570469  0.252412  0.993433  0
0  0.139853  0.268120  0.382930  0.303612  0
3  0.350653  0.228341  0.262602  0.720907  1
8  0.792669  0.890965  0.750801  0.382474  1
0  0.139853  0.268120  0.382930  0.303612  0
7  0.613138  0.847137  0.668122  0.882749  1

### 调参与最终模型

大多数学习算法都有些参数（parameter）需要设定，参数配置不同，学得模型的性能往往有显著差别。

在选择模型评估与选择时，除了要对使用学习算法进行选择，还需要对算法参数进行设定，这就是通常所说的**“参数调节”或简称“调参”（parameter tuning）**。

有些同学可能想到，调参和算法选择没什么本质区别：对每种参数配置都训练出模型，然后把对应最好模型的参数作为结果。这个认识基本正确，但要注意：学习算法的好多参数是在实数范围内取值，因此，对每种参数配置都训练出模型来说是不行的。

**现实中常用的办法是对每个参数选定范围和变化步长。**例如在[0,1]范围内，以0.1为步长，所以实际要评估的参数是5个数字，最终结果从这5个中选出。

事实上，即便是在计算开销和性能估计之间进行折中，调参也很困难。例如：某个算法有3个参数，每个参数仅考虑5个候选值，这样每组训练/测试集就有$5^3 = 125$个模型需要考察。很多强大的学习算法有大量参数需要设定，这将导致极大的调参工程量。

在训练和测试完成后，模型选择完成，学习算法和参数配置基本确定，此时需要重新用数据集D的全集进行重新训练，对参数进行微调，最后确定的模型才是交付用户的模型。

- 通常，我们把学得模型在实际中遇到的数据称为**测试数据。**
- 为加以区分，模型评估与选择中用于评估测试的数据称为**“验证集”**

人们使用测试数据来比较各种算法，使用训练集和验证集来进行模型选择和调参。

## 性能度量

对学习器的泛化性能进行评估，不仅需要有效可行的实验估计方法，还需要有衡量模型泛化能力的评价标准，这既是**性能度量（performance measure）**。

性能度量反映了任务需求，在对比不同模型的能力时，使用不同的性能度量往往会有不同的评判结果。这意味着模型的“好坏”是相对的。什么样的模型是好的，不仅取决于算法和数据，还取决于任务需求。

在预测任务中，给定样例集$D={(x_1,y_1),(x_2,y_2),...,(x_m,y_m)}$，其中$y_i$是示例$x_i$的真实标记。

要评估学习器$f$的性能，就要把学习器预测结果$f(x)$与真实标记$y$进行比较。

### 错误率与精度

#### 回归任务性能度量

回归任务中，最常用的性能度量是“均方误差”（mean squared error)

$E(f;D) = \frac {1}{m}\sum_{i=1}^{m}(f(x_i)-y_i)^2$   ——式（2）

更一般地，对于数据分布$D$和概率密度函数$p(.)$，均方误差可描述为：

$E(f;D) = \int _{x \in D}(f(x) - y)^2p(x)dx$   ——式（3）

#### 分类任务性能度量

分类任务中最常用的两种性能度量是：

- 错误率：分类错误的样本数占样本总数的比例；
- 精度：分类正确的样本数占样本总数的比例。

这两个度量适用于二分类任务，也适用于多分类任务。

对于样例集D，分类错误率定义为：

$E(f;D) = \frac {1}{m}\sum_{i=1}^{m}II(f(x_i) \neq y_i)$   ——式（4）

精度则定义为：

$acc(f;D) = \frac {1}{m}\sum_{i=1}^{m}II(f(x_i) = y_i) = 1 - E(f;D)$   ——式（5）

更一般的，对于数据分布D和概率密度函数p(.)，错误率与精度可分别描述为：

$E(f;D) = \int_{x \in D}II(f(x_i) \neq y_i)p(x)dx $   ——式（6）

$acc(f;D) = \int_{x \in D}II(f(x) = y)p(x)dx  = 1 - E(f;D)$   ——式（7）

### 查准率（precision）、查全率（recall）与F1

错误率和精度虽常用，但不能满足所有任务需求。

例如，瓜农拉来一车西瓜，我们用训练好的模型对这些西瓜进行判别，显然，错误率衡量了有多少比例的瓜被判别错误。但我们关心的是“挑出的西瓜中有多少比例是好瓜？”或者“所有好瓜中有多少比例被挑了出来？”那么错误率显然就不够用了。

类似需求在信息检索、web搜索中常见，例如信息检索中，较为关心“检索出的信息中有u瓯绍比例是用户感兴趣的？”、“用户感兴趣的信息中有多少被检索出来了？”

“查全率”和“查准率”是更为适用于此类需求的性能度量。

对于二分类问题，可以将样例根据其真实类别与学习器预测类别的组合划分为：

- 真正例（true positive，TP）
- 假正例（false positive,FP)
- 真反例（true negative,TN）
- 假反例（false negative,FN)

|+|预测为正例|预测为反例|
|-|-|-|
|真实正例|真正例 TP|假反例 FN|
|真实反例|假正例FP|真反例TN|

查准率P与查全率R分别定义为：

查准率 $P = \frac {TP}{TP+FP}$   ——式（8）

查全率 $R = \frac {TP}{TP+FN}$   ——式（9）

查准率和查全率是一对矛盾的度量。一般来说，查准率高时，查全率往往偏低；而查全率高时，查准率往往偏低。

#### P-R图

很多情况下，我们可以根据学习器的预测结果对样例进行排序：

- 排在最前面的，是学习器认为“最可能”是正例的样本。
- 排在最后面的，是学习器认为“最不可能”是正例的样本。

按照次顺序，逐个把样本作为正例进行预测，则每次可以计算出当前的查全率、查准率。以查准率为纵轴、查全率为横轴作图，就可以得到查准率-查全率曲线，简称“P-R曲线”，显示该曲线的图成为“P-R图”。

![PR曲线与平衡点示意图](images\模型评估\PR曲线与平衡点示意图.png)

P-R图直观地显示出学习器在样本总体上地查全率、查准率。

在进行比较时，若一个学习器地P-R曲线被另一个学习器地曲线完全“包住”，则可断言，后者地性能由于前者。例如上图中A的性能优于C；如果两个学习器的P-R曲线发生了交叉，例如A和B，则难以简单断言。如果一定要比较，可以比较P-R曲线下面积的大小，它在一定程度上表征了学习器在查全率和查准率上取得相对“双高”的比例。

但是这个面积值不易估算，所以，人们设计了一些综合考查查准率、查全率的性能度量。

- 平衡点（Break-Event Point，简称BEP）

是一个综合考查查准率、查全率的性能度量，它是“查全率=查准率”时的取值。上图中学习器C的BEP=0.64，使用BEP比较，可以认为A优于B。

但BEP过于简化，更常用的是F1度量.

- F1度量

$F1 = \frac{2 \times P \times R}{P + R} = \frac {2 \times TP}{样例总数 + TP - TN}$ ——式（10）

F1是基于查准率与查全率的调和平均（harmonic mean）定义的：

$ \frac{1}{F1} =  \frac{1}{2} \times (\frac{1}{P}+\frac{1}{R})$

与算术平均$\frac{P+R}{2}$和几何平均$\sqrt{P \times R}$ 相比，调和平均更重视较小的值。

在一些应用中，对查全率和查准率的重视程度有所不同。例如，在商品推荐系统中，为了尽可能少打扰用户，更希望推荐内容确实是用户感兴趣的那类，此时查准率更为重要。而在逃犯信息检索系统中，更希望尽可能少的漏掉逃犯，此时查全率更加重要。

F1度量的一般形式：$F^\beta$，能让我们表达出对查准率/查全率的不同偏好，他定义为：

$F_\beta = \frac {(1+\beta^2) \times P \times R}{(\beta^2 \times P} + R$  ——式（11）

其中，$\beta > 0 $度量了查全率对查准率的相对重要性。$\beta = 1$时退化为标准的F1;

$\beta > 1 $时查全率有更大影响；$\beta < 1 $时查准率有更大影响；

$F_\beta$是加权调和平均：

$\frac{1}{F_\beta} = \frac{1}{1+\beta^2} \times (\frac{1}{P}+\frac{\beta^2}{R})^2$

很多时候，我们有多个二分类混淆矩阵，例如进行多次训练/测试，每次得到一个混淆矩阵；或是在多个数据集上进行训练/测试，希望估计算法的“全局”性能；甚或是执行多分类任务，每两两类型的组合都对应一个混淆矩阵；总之，我们希望在n个二分类混淆矩阵上综合考查查全率和查准率。

一种直接的做法时现在各混淆矩阵上分别计算出查准率和查全率，记为$(P_1,R_1),(P_2,R_2),...,(P_n,R_n)$，在计算平均值，这样就得到“宏查准率”（marco-P）、“宏查全率”（marco-R），以及相应的“宏F1”（marco-F1）：

$marco-P = \frac{1}{n}\sum_{i=1}^{n}P_i$ ——式（12）

$marco-R = \frac{1}{n}\sum_{i=1}^{n}R_i$ ——式（13）

$marco-F1 = \frac{2 \times macro-P \times marco-R}{marco-P + marco-R}$ ——式（14）

还可先将各混淆矩阵的对应元素进行平均，得到TP、FP、TN、FN的平均值，分别记为$\overline{TP}、$\overline{FP}、$\overline{TN}、$\overline{FN}$，在基于这些平均值计算出“为查准率”（micro-P）、“微查全率”（micro-R）和“微F1”（mircro-F1）：

$micro-P = \frac{\overline{TP} }{\overline{TP} + \overline{FP}}$ ——式（12）

$micro-R = \frac{\overline{TP} }{\overline{TP} + \overline{FN}}$ ——式（13）

$micro-F1 = \frac{2 \times micro-P \times micro-R}{micro-P + micro-R}$ ——式（14）


### ROC和AUC

很多学习器是为测试岩本禅师一个实值或概率预测值，然后将这个预测值与一个分类阈值（threshold）进行比较，若大于阈值则分为正类，否则为反类。

例如，神经网络在一般情形下是对每个测试样本预测出一个[0.0 ,1.0]之间的实值，然后将这个值与0.5进行比较，大于0.5则判为正例，否则为反例。

这个实值或概率预测结果的好坏，直接决定了学习器的泛化能力。

实际上，根据这个实值或概率预测结果，我们可将测试样本进行排序，“最可能”是正例的排在最前面，“最不能”是正例的排在最后面。这样，分类过程就相当于宰割排序中以某个“截断点”（cut point）将样本分为两部分，前一部分判为正例，后一部分判为反例。

在不同的应用任务中，我们可根据任务需求来采用不同的截断点，例如，若我们更重视“查准率”，则可选择排序中考前的位置进行截断；若更重视“查全率”则可选择靠后的位置进行截断。因此，排序本身的质量好坏，体现了综合考虑学习器在不同任务下的“期望泛化性能”的好坏，或者说，“一般情况下”泛化性能的好坏，ROC曲线则是从这个角度出发来研究学习器泛化性能的有力工具。

ROC全称是“受试者工作特征”（Receiver Operating Characteristic）曲线。

它源于二战中用于敌机检测的雷达信号分析，后来被应用于心理学、医学、机器学习。与P-R曲线类似，我们根据学习器的预测结果对样例进行排序，按此顺序，逐个把样本作为正例进行预测，二米此计算出两个重要量的值，分别以它们为横、纵坐标作图，就得到了“ROC曲线”。

与P-R曲线使用查准率、查全率为纵、横轴不同，ROC曲线的纵轴是“真正例率”（True Positive Rate，简称TPR）,横轴是“假正例率”（False positive Rate，简称FPR）。

真正例率：指真正例样本数量，与实际为正例的样本的比值。

假正例率：指假正例样本数量，与实际为反例的样本的比值。

真正例率 $TPR = \frac{TP}{TP+FN}$  ——式（18）

> 查全率 $R = \frac {TP}{TP+FN}$

假正例率 $FPR = \frac{FP}{TN+FP}$  ——式（18）

> 查准率 $P = \frac {TP}{TP+FP}$

显示ROC曲线的图称为ROC图。下图给出了示意。显然对角线对应于应于“随机猜测”模型，而点(0,1)则对应于将所有正例排在所有反例之前的“理想模型”。

![ROC曲线与AUC示意图](images\模型评估\ROC曲线与AUC示意图.png)

现实任务中，通常是利用有限个测试样例来绘制ROC图，此时仅能获得有限个（真正例率，假正例率）坐标对，无法产生上图的光滑曲线。绘图过程很简单：

- 给定$m^+$个正例和$m^-$个反例，根据学习器预测结果对样例进行排序，然后把分类阈值设为最大 ，即把所有样例均预测为反例，此时真正例率和假正例率全部为0，在坐标(0,0)处标记一个点。

- 然后，将分类阈值，依次设为每个样例的预测值，即依次将每个样例划分为正例。设前一个标记点坐标为(x,y)，当前若为真正例，则对应标记点的坐标为$(x,y+\frac{1}{m^+})$；当前若为假正例，则对应标记点的坐标为$（x+\frac{1}{m^-},y）$,然后用线段连接相邻点即得。

- 如果有预测值相同的的多个样例，其中若有a个真正例，b个假正例，那么这些样例作为一个点标记在图上，坐标为$（x+\frac{b}{m^-},y+\frac{a}{m^+}）$

进行学习器的比较时，与P-R图相似，若一个学习器的ROC曲线被另一个学习器的曲线完全“包住”，则可断言后者的性能优于前者。

若两个学习器的ROC曲线发生交叉，则难以断言孰优孰劣。较为合理的比较方法是比较ROC曲线下的面积，即AUC（Area Under ROC Curve），如上图中图所示。

从定义可知，AUC可通过对ROC曲线下各部分的面积求和而得。假定ROC曲线是由坐标为${(x_1,y_1)(x_2,y_2),...,(x_m,y_m)}$的点按序连接而形成${(x_1 = 0,x_m = 1)$,则AUC可估算为：

$AUC = \frac{1}{2}\sum_{i=1}^{m-1}(x_{i+1} - x_i)(y_i +y_{i+1})$  ——式（20）


#### AUC的计算

AUC越大，模型越好，随机模型的AUC为0.5，理想模型的AUC=1.

如何计算AUC的值呢？

- 早期方法，计算ROC曲线下面积。

这是最直观的，根据AUC这个名称，我们知道，计算出ROC曲线下面的面积，就是AUC的值。事实上，这也是在早期 Machine Learning文献中常见的AUC计算方法。由于我们的测试样本是有限的。我们得到的AUC曲线必然是一个阶梯状的。因此，计算的AUC也就是这些阶梯下面的面积之和。但后来认为这种方法比较麻烦。

- 计算损失函数，然后用公式计算

一个关于AUC的很有趣的性质是，它和Wilcoxon-Mann-Witney Test是等价的。这个等价关系的证明留在下篇帖子中给出。而Wilcoxon-Mann-Witney Test就是测试任意给一个正类样本和一个负类样本，正类样本的score有多大的概率大于负类样本的score。有了这个定义，我们就得到了另外一中计 算AUC的办法：得到这个概率。我们知道，在有限样本中我们常用的得到概率的办法就是通过频率来估计之。这种估计随着样本规模的扩大而逐渐逼近真实值。这 和上面的方法中，样本数越多，计算的AUC越准确类似，也和计算积分的时候，小区间划分的越细，计算的越准确是同样的道理。具体来说就是统计一下所有的 M×N(M为正类样本的数目，N为负类样本的数目)个正负样本对中，有多少个组中的正样本的score大于负样本的score。当二元组中正负样本的 score相等的时候，按照0.5计算。然后除以MN。实现这个方法的复杂度为O(n^2)。n为样本数（即n=M+N） 

形式化来看，AUC考虑的是样本预测的排序质量，因此他与排序误差有紧密联系。给定$m^+$个正例和$m^-$个反例，令$D^+$和$D^-$分别表示正、反例集合，则排序“损失”（loss）定义为：

$l_{rank} = \frac{1}{m^+ m^-}\sum_{x^+ \in D^+}\sum_{x^- \in D^-}(II(f(x^+)<f(x^-)) + \frac{1}{2}II(f(x^+)=f(x^-)))$  ——式（21）

即考虑每一对正例、反例，若正例的预测值小于反例，则记一个“罚分”，若相等，则记0.5个“罚分”。

容易看出，$l_{rank}$对应的是ROC曲线值上的面积：

若一个正例在ROC曲线上对应标记点的坐标为$(x,y)$，则x恰是排序在其之前的反例所占的比例（按照上述作图法，x取值为x个$\frac{1}{m^-},即之前出现的反例所占比例$），即假正例率。因此有：

$AUC = 1 - l_{rank}$  ——式（22）

所以说，AUC值是一个概率值，当你随机挑选一个正样本和任一负样本，当前的分类算法根据计算得到的Score值将这个正样本排在负样本前面的概率就是AUC值，AUC值越大，当前分类算法越有可能将正样本排在负样本前面，从而能够更好地分类。

AUC还常常被用来作为模型排序好坏的指标，原因在于AUC可以看做随机从正负样本中选取一对正负样本，其中正样本的得分大于负样本的概率！

- 第三种方法，使用下列公式

第三种方法实际上和上述第二种方法是一样的，但是复杂度减小了。它也是首先对score从大到小排序，然后令最大score对应的sample 的rank为n，第二大score对应sample的rank为n-1，以此类推。然后把所有的正类样本的rank相加，再减去$m^+$个正样中两两组合的个数。得到的就是所有的样本中有多少对正类样本的score大于负类样本的score，然后再除以$m^+ \times m^-$。即 


$AUC = \frac {\sum_{i \in m^+}rank_i - \frac{(m^+-1)m^+}{2}}{m^+ \times m^-} = \frac{n-m^+ +1}{m^-}$






公式解释：

1.为了求的组合中正样本的score值大于负样本，如果所有的正样本score值都是大于负样本的，那么第一位与任意的进行组合score值都要大，我们取它的rank值为n，但是n-1中有M-1是正样例和正样例的组合这种是不在统计范围内的（为计算方便我们取n组，相应的不符合的有M个），所以要减掉，那么同理排在第二位的n-1，会有M-1个是不满足的，依次类推，故得到后面的公式M*(M+1)/2，我们可以验证在正样本score都大于负样本的假设下，AUC的值为1

2.根据上面的解释，不难得出，rank的值代表的是能够产生score前大后小的这样的组合数，但是这里包含了（正，正）的情况，所以要减去这样的组（即排在它后面正例的个数），即可得到上面的公式

另外，特别需要注意的是，再存在score相等的情况时，对相等score的样本，需要 赋予相同的rank(无论这个相等的score是出现在同类样本还是不同类的样本之间，都需要这样处理)。具体操作就是再把所有这些score相等的样本 的rank取平均。然后再使用上述公式。 
